### 导包

In [1]:
import utils.util as util
import json
import numpy as np
import utils.visualization as visual
from tqdm import tqdm

### 涉及阈值

In [2]:
DistThreh = 50
TimeThreh = 180

### 停留点类

In [3]:
class Stay_point:
    def __init__(self, center, point_list, arrive_time, leave_time):
        self.center = center
        self.point_list = point_list
        self.arrive_time = arrive_time
        self.leave_time = leave_time

### 支持函数

In [4]:
# 读取轨迹数据 并将轨迹坐标系转换为墨卡托坐标系
def load_trajectories(filename):
    raw_traj = json.load(open(filename))
    for k, v in raw_traj.items():
        raw_traj[k] = [v[0], util.transform_points_wgs84_to_mercator(v[1])]
    return raw_traj

### 基于距离与时间阈值的停留点检测

In [5]:
def extract_stay_point_for_single_traj(traj):
    stay_points = []
    times, coors = traj
    i = 0
    length = len(coors)
    while i < length:
        j = i + 1
        while j < length:
            distance = util.eucl_distance(coors[i], coors[j])
            if distance > DistThreh:
                time = times[j] - times[i]
                if time > TimeThreh:
                    center = np.array(coors[i : j + 1]).mean(0).tolist()
                    stay_points.append(Stay_point(center, coors[i : j + 1], times[i], times[j]))
                    i = j
                else:
                    i += 1
                break
            j += 1
        if j == length:
            break

    return stay_points

def extract_stay_points_for_all_traj(raw_traj):
    stay_points = []
    for k, v in tqdm(raw_traj.items()):
        stay_points.extend(extract_stay_point_for_single_traj(v))
    return stay_points

## 读入数据，调用停留点识别算法

#### 数据输入
- traj_dict：轨迹dict
- key：轨迹id
- value：[[时间戳列表],[经纬度点列表]]

#### 调用停留点提取算法的输出结果
- stay_points：停留点list，内部每一项为停留点类，每个停留点包含center, point_list, arrive_time, leave_time属性

In [6]:
# 读取数据
data = load_trajectories('porto_traj.json')

In [7]:
# 数据样例
print(data['0'])

[[1372636858, 1372636859, 1372636869, 1372636870, 1372636871, 1372636872, 1372636873, 1372636877, 1372636878], [[-960588.13146654, 5034671.201921352], [-960590.1352173749, 5034675.193587552], [-960552.063951513, 5034817.564090121], [-960541.0433219211, 5034842.845052223], [-960549.0583252608, 5034864.134334601], [-960779.4896712671, 5035158.197331414], [-960779.4896712671, 5035159.527953491], [-960779.4896712671, 5035156.866709527], [-960780.4915466846, 5035155.536087831]]]


In [8]:
# 停留点提取
stay_points = extract_stay_points_for_all_traj(data)

100%|██████████| 59378/59378 [00:10<00:00, 5760.20it/s]


In [10]:
stay_points[0]

In [11]:

print(stay_points[0].center)
print(stay_points[0].point_list)
print(stay_points[0].arrive_time)
print(stay_points[0].leave_time)

[-964855.3010284916, 5039254.475553829]
[[-964836.083236394, 5039238.743375926], [-964848.1057414027, 5039249.392841645], [-964847.1038659858, 5039249.392841645], [-964856.1207447423, 5039257.379948615], [-964853.11511849, 5039256.048763657], [-964866.1394989162, 5039269.360621423], [-964858.1244955772, 5039256.048763657], [-964842.0944888983, 5039236.081011324], [-964849.1076168203, 5039241.405741261], [-964870.147000586, 5039265.367062174], [-964882.169505595, 5039280.010120798]]
1372654104
1372654435


In [12]:
# 可视化停留点中心位置
visual.visual_traj(util.transform_points_mercator_to_wgs84([i.center for i in stay_points]), 'stay_centers.json')